<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Using_an_LSTM_Autoencoder_for_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed
from tensorflow.keras.models import Sequential
import numpy as np

# Generate synthetic time-series data
num_samples = 1000  # Increase the number of samples
timesteps = 50
features = 1

# Normal data generation (sine wave)
normal_data = np.array([np.sin(np.linspace(0, 100, timesteps)) for _ in range(num_samples)]).reshape(num_samples, timesteps, features)

# Anomalous data generation (adding noise)
anomalous_data = normal_data + np.random.normal(0, 0.5, normal_data.shape)

# Normalize the data (optional but recommended)
normal_data_mean = np.mean(normal_data)
normal_data_std = np.std(normal_data)
normal_data_normalized = (normal_data - normal_data_mean) / normal_data_std
anomalous_data_normalized = (anomalous_data - normal_data_mean) / normal_data_std

# Define an LSTM autoencoder model
model = Sequential([
    tf.keras.Input(shape=(timesteps, features)),  # Define input shape using Input layer
    LSTM(64, activation='relu', return_sequences=True),
    LSTM(32, activation='relu', return_sequences=False),
    RepeatVector(timesteps),
    LSTM(32, activation='relu', return_sequences=True),
    LSTM(64, activation='relu', return_sequences=True),
    TimeDistributed(tf.keras.layers.Dense(features))
])

# Compile and fit model (training only on normal data)
model.compile(optimizer='adam', loss='mse')
model.fit(normal_data_normalized, normal_data_normalized, epochs=50, batch_size=16, validation_split=0.1)

# Detect anomalies by thresholding reconstruction error
reconstructed = model.predict(anomalous_data_normalized)
mse = np.mean(np.power(anomalous_data_normalized - reconstructed, 2), axis=(1, 2))

# Dynamic thresholding based on mean and standard deviation of MSE
threshold_mean = np.mean(mse)
threshold_std = np.std(mse)
dynamic_threshold = threshold_mean + 2 * threshold_std  # Set threshold as mean + 2*std

# Identify anomalies
anomalies = mse > dynamic_threshold

# Print results
print("Reconstruction error (MSE):", mse)
print("Dynamic Threshold:", dynamic_threshold)
print("Anomalies detected:", anomalies)

# Optional: Count and display number of anomalies detected
num_anomalies_detected = np.sum(anomalies)
print(f"Number of anomalies detected: {num_anomalies_detected} out of {len(anomalies)} samples")